# Assignment 2

# Task 1

Neural networks....................... Generative

Naive Bayes classifier................ Discriminative

Logistic regression................... Generative

Gaussian Mixture model................ Generative

GANs.................................. Generative

LDA(Latent Dirichlet Allocation)...... Generative

SVM................................... Discriminative

Decision Tree......................... Discriminative

# Task 2

In [1]:
from nltk.corpus import treebank,brown
import numpy as np

In [2]:
corpus = brown.tagged_sents(tagset='universal')[:-100]
test_corpus = brown.tagged_sents(tagset='universal')[-10:]

In [3]:
pos_dict = {}
pos_ord = []
word_dict = {}
for i in corpus:
    pos_ord.append([j[1] for j in i])
    for j in i:
        if j[1] not in pos_dict:
            pos_dict[j[1]] = [j[0].lower()]
        else:
            pos_dict[j[1]].append(j[0].lower())
        if j[0].lower() not in word_dict:
            word_dict[j[0].lower()] = [j[1]]
        else:
            word_dict[j[0].lower()].append(j[1])

#### Transition Matrix

In [4]:
transition_mat = {i:{j:0 for j in pos_dict} for i in pos_dict}

In [5]:
for i in corpus:
    for j in range(len(i)-1):
        transition_mat[i[j][1]][i[j+1][1]] += 1

In [6]:
transition_mat = {i:{j:transition_mat[i][j]/sum(transition_mat[i].values()) for j in pos_dict} for i in pos_dict}

In [7]:
TM = np.array([list(i.values()) for i in transition_mat.values()])

In [8]:
TM

array([[  5.91780902e-03,   6.26468480e-01,   2.39835852e-01,
          6.47155209e-02,   9.09250508e-03,   1.27499890e-02,
          1.75047182e-02,   6.43717174e-04,   2.01161617e-03,
          9.89715155e-03,   9.76548213e-03,   1.39715887e-03],
       [  1.55527916e-02,   1.49845165e-01,   1.29375590e-02,
          1.59437999e-01,   2.45189905e-01,   2.84527817e-01,
          2.64587125e-02,   5.98767886e-02,   1.78871693e-02,
          1.98495056e-02,   8.10466766e-03,   3.31919333e-04],
       [  5.85278276e-03,   6.53058049e-01,   5.69239976e-02,
          1.74745661e-02,   8.84739677e-02,   1.00371035e-01,
          9.64691801e-03,   3.76181927e-02,   1.92938360e-02,
          3.80610413e-03,   6.98982645e-03,   4.90724117e-04],
       [  1.63020222e-01,   9.76256048e-02,   5.75714019e-02,
          1.84405139e-01,   1.69274405e-01,   8.06515323e-02,
          1.03320203e-01,   1.43846213e-02,   6.56305204e-02,
          5.49545200e-02,   8.98078759e-03,   1.81042144e-04],
    

#### Start Matrix

In [9]:
start_pos = [i[0] for i in pos_ord]
start_mat = {i:start_pos.count(i)/len(start_pos) for i in pos_dict.keys()}

In [10]:
SM = np.array(list(start_mat.values()))

In [11]:
SM

array([ 0.21339972,  0.14129979,  0.03434661,  0.04509085,  0.12283368,
        0.08901118,  0.0911775 ,  0.04916143,  0.03665269,  0.15971349,
        0.01678896,  0.00052411])

#### Emission Matrix

In [12]:
emission_matrix = {i:{j:0 for j in word_dict} for i in pos_dict}

In [13]:
for i in corpus:
    for j in i:
        emission_matrix[j[1]][j[0].lower()] += 1

In [14]:
emission_matrix = {i:{j:emission_matrix[i][j]/len(pos_dict[i]) for j in word_dict} for i in pos_dict}

In [15]:
EM = np.array([list(i.values()) for i in emission_matrix.values()])

In [16]:
EM

array([[  5.10846669e-01,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   6.18013269e-05,   5.59847314e-04, ...,
          3.63537217e-06,   0.00000000e+00,   3.63537217e-06],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   1.19644417e-05,   0.00000000e+00],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.19138057e-03,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

### Viterbi algorithm

In [17]:
import random

def viterbi(observations, pos_names = list(pos_dict.keys()), start_probability= start_mat, transition_probability = transition_mat, emission_probability= emission_matrix):
    pos = [{}]
    path = {}
    
    r = random.uniform(0,1e-06)
    for i in observations:
        for j in pos_names:
            if i not in emission_matrix[j]:
                emission_matrix[j][i] = r
    
    for state in pos_names:
        pos[0][state] = start_probability[state] * emission_probability[state][observations[0]]
        path[state] = [state]

    for observations_index in range(1,len(observations)):
        pos.append({})
        new_path = {}
        for state in pos_names:
            (probability, possible_state) = max(
            [(pos[observations_index-1][y0] * transition_probability[y0][state] 
            * emission_probability[state][observations[observations_index]], y0) for y0 in pos_names])

            pos[observations_index][state] = probability
            new_path[state] = path[possible_state] + [state]

        path = new_path

    (probability, state) = max([(pos[len(observations) - 1][state], state) for state in pos_names])
    return probability, path[state]

In [18]:
def accu(a, b):
    return sum(1 for x,y in zip(a,b) if x == y) / len(a)

#### Testing

In [19]:
print('For the following corpus:\n')
print(' '.join([i[0] for i in corpus[0]]))
print(' '.join([i[1] for i in corpus[0]]),'-True')
print(' '.join(viterbi([i[0].lower for i in corpus[0]])[1]),'-predicted')

For the following corpus:

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .
DET NOUN NOUN ADJ NOUN VERB NOUN DET NOUN ADP NOUN ADJ NOUN NOUN VERB . DET NOUN . ADP DET NOUN VERB NOUN . -True
DET NOUN ADP DET NOUN ADP DET NOUN ADP DET NOUN ADP DET NOUN ADP DET NOUN ADP DET NOUN ADP DET NOUN ADP DET -predicted


#### On Test Corpus

In [21]:
sum([accu([j[1] for j in i], viterbi([j[0].lower() for j in i])[1]) for i in test_corpus])/len(test_corpus)

0.923871132215036

# Task 3

## CRF

In [22]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [23]:
def word2features(sent,i):
    word = sent[i][0]
    
    features ={
    'bias': 1.0,
    }
                
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for i,label in sent]

In [24]:
X_train=[sent2features(s) for s in corpus]
y_train=[sent2labels(s) for s in corpus]

X_test=[sent2features(s) for s in test_corpus]
y_test=[sent2labels(s) for s in test_corpus]

In [27]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True)

In [28]:
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [29]:
y_pred = crf.predict(X_test)
labels=list(crf.classes_)

metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.12985271687027178

In [30]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

          .      0.800     0.242     0.372        33
          X      0.023     1.000     0.044         3
        ADJ      0.000     0.000     0.000        18
        ADP      0.179     0.185     0.182        27
        ADV      0.000     0.000     0.000         9
       VERB      0.000     0.000     0.000        35
        DET      0.121     0.121     0.121        33
       CONJ      0.000     0.000     0.000         7
       NOUN      0.242     0.157     0.190        51
       PRON      0.000     0.000     0.000        12
        PRT      0.000     0.000     0.000        11
        NUM      0.000     0.000     0.000         0

avg / total      0.199     0.117     0.130       239



/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
